In [1]:
import neptune

neptune.init(
    project_qualified_name='arsde/J21SYSU',
)

neptune.create_experiment()#'JSYSU-3')

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/arsde/J21SYSU/e/JSYSU-67


Experiment(JSYSU-67)

In [2]:
import os


def init(run_name):
    os.system('mkdir {}'.format(run_name))
    os.system('mkdir {}/models_saved'.format(run_name))
    os.system('mkdir {}/results'.format(run_name))
#     os.system('mkdir {}/preds'.format(run_name))
    
run_name = input()
init(run_name)

JSYSU-67


In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import plotly.io as pio
pio.templates.default = 'plotly_white'

import warnings
warnings.filterwarnings("ignore")

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17579420690435562310
]


In [5]:
bdt_J19 = pd.read_csv('../df_bdt_eval.csv')
bdt_J19_5M_23 = bdt_J19[(bdt_J19['opt']==23) & (bdt_J19['model']=='5M')]
bdt_J19_5M_0 = bdt_J19[(bdt_J19['opt']==0) & (bdt_J19['model']=='5M')]

In [6]:
path='/mnt/cephfs/ml_data/mc_2021/processed_data'
data_real = pd.read_csv(f'{path}/ProcessedTrainReal/ProcessedTrain.csv.gz')
data_real = data_real[data_real['edepR'] < 17.2]

In [7]:
features_opt = ['AccumCharge', 'nPMTs', 'R_cc', 'R_cht', 'pe_mean',
                'pe_std', 'pe_skew', 'pe_kurtosis', 'pho_cc', 'pho_cht',
                'ht_2p', 'ht_5p', 'ht_10p', 'ht_15p', 'ht_20p',
                'ht_25p', 'ht_30p', 'ht_35p', 'ht_40p', 'ht_45p',
                'ht_50p', 'ht_55p', 'ht_60p', 'ht_65p', 'ht_70p',
                'ht_75p', 'ht_80p', 'ht_85p', 'ht_90p', 'ht_95p']

In [8]:
len(features_opt)

30

In [9]:
all_data = [data_real]

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

X = data_real.iloc[:, :-5][features_opt]
scaler.fit(X)

X = scaler.transform(X)
y = data_real['edep']

In [11]:
X.shape

(4604438, 30)

In [12]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

[]

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, concatenate, Input
from tensorflow.keras.layers import Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import CosineDecay, ExponentialDecay

kernel_initializer_input = ['normal'] #['normal', 'lecun_normal', 'uniform']
kernel_initializer_hidden = ['normal'] #['normal', 'lecun_normal', 'uniform']
activations = ['relu'] #['relu', 'elu', 'selu']
optimizers = ['SGD'] # ['adam', 'rmsprop', 'SGD']
schedules = ['None', 'ExponentialDecay']
units_input_list = [256] # [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
units_in_hidden_layer_list = [256] # [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
num_hidden_layers_list = [16] # [1, 2, 4, 8, 16, 32]

def build_model(hp):
    input_features = Input(shape=X.shape[1])
    
#     units_input = hp.Int('units_input', min_value=16, max_value=256, default=32, step=8)
#     units_in_hidden_layer = hp.Int('units_in_hidden_layer', min_value=16, max_value=256, default=32, step=8)
#     num_hidden_layers = hp.Int('num_hidden_layers', min_value=1, max_value=16, default=2, step=1)

    units_input = hp.Choice('units_input', units_input_list, default=256)
    units_in_hidden_layer = hp.Choice('units_in_hidden_layer', units_in_hidden_layer_list, default=256)
    num_hidden_layers = hp.Choice('num_hidden_layers', num_hidden_layers_list, default=16)
    kernel_initializers_input = hp.Choice('kernel_initializers_input', kernel_initializer_input)  
    kernel_initializers_hidden = hp.Choice('kernel_initializers_hidden', kernel_initializer_hidden)
    activation = hp.Choice('activation', values=activations)      
    optimizer = hp.Choice('optimizer', values=optimizers)
    
    lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, default=1e-3, sampling='LOG')
    decay_lr = hp.Choice('decay_lr', schedules)
        
#     batch_norm = hp.Choice('batch_norm', [False, True])
       
    if decay_lr == 'ExponentialDecay':
        decay_steps = hp.Int('decay_steps', min_value=500, max_value=20000, default=2000)
        decay_rate = hp.Float('decay_rate', min_value=0.1, max_value=0.9, default=0.8)

        lr = ExponentialDecay(
            initial_learning_rate=lr,
            decay_steps=decay_steps,
            decay_rate=decay_rate
        )
    elif decay_lr == 'CosineDecay':
        decay_steps = hp.Int('decay_steps', min_value=500, max_value=20000, default=2000)

        lr = CosineDecay(
            initial_learning_rate=lr,
            decay_steps=decay_steps,
        )

    x = Dense(units=units_input,
              kernel_initializer=kernel_initializers_input,
              activation=activation
            )(input_features)
#     if batch_norm:
#         x = BatchNormalization()(x)

    for i in range(num_hidden_layers):
        x = Dense(units=units_in_hidden_layer,
                  kernel_initializer=kernel_initializers_hidden,
                  activation=activation
            )(x)
#         if batch_norm:
#             x = BatchNormalization()(x)

    output = Dense(units=1,
                   kernel_initializer="normal",
                   activation='linear')(x)

    model = Model(inputs=input_features,
                  outputs=output,
                  name='Model')

#     if optimizer == 'adam':
#         beta_1 = hp.Float('beta_1', min_value=0.01, max_value=1, default=0.9)
#         beta_2 = hp.Float('beta_2', min_value=0.01, max_value=1, default=0.999)
#         optimizer = Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2)
#     elif optimizer == 'rmsprop':
#         rho = hp.Float('rho', min_value=0.01, max_value=1, default=0.9)
#         momentum = hp.Float('momentum', min_value=0.0, max_value=1, default=0.9)
#         optimizer = RMSprop(learning_rate=lr, rho=rho, momentum=momentum)
#     elif optimizer == 'SGD':
    momentum = hp.Float('momentum', min_value=0.0, max_value=1, default=0.9)
    nesterov = hp.Boolean('nesterov', default=True)
    optimizer = SGD(learning_rate=lr, momentum=momentum, nesterov=nesterov)
    
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.MeanAbsolutePercentageError(),
        metrics=[
                tf.keras.metrics.MeanAbsolutePercentageError(name='mape'),
                'mse',
                'mae'
            ])
    
    model.summary()
    return model

In [14]:
import neptunecontrib.monitoring.kerastuner as npt_utils
from kerastuner.tuners import BayesianOptimization, Hyperband


# class MyTuner(BayesianOptimization):
#     def run_trial(self, trial, *args, **kwargs):
#         kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 256, 2048, step=128)
#         super(MyTuner, self).run_trial(trial, *args, **kwargs)

seed = np.random.randint(10000)
print(seed)

tuner = BayesianOptimization(
    build_model,
    objective='val_mape',
    max_trials=60,
    seed=seed,
    directory='saved_networks',
    project_name=run_name,
    logger=npt_utils.NeptuneLogger()
)

7525
Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               6

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

monitor = EarlyStopping(monitor='val_mape', patience=5, mode='min',
        restore_best_weights=True)

In [ ]:
tuner.search(X,
             y,
             batch_size=1024,
             epochs=200, 
             validation_split=0.1,
             callbacks=[monitor],
             verbose=2
)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units_input       |256               |?                 
units_in_hidden...|256               |?                 
num_hidden_layers |16                |?                 
kernel_initiali...|normal            |?                 
kernel_initiali...|normal            |?                 
activation        |relu              |?                 
optimizer         |SGD               |?                 
lr                |0.0001058         |?                 
decay_lr          |None              |?                 
momentum          |0.48856           |?                 
nesterov          |True              |?                 

Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (De

In [ ]:
tuner.results_summary()

In [ ]:
npt_utils.log_tuner_info(tuner)

In [ ]:
model = tuner.get_best_models()[0]
model.save("{0}/models_saved/{0}.h5".format(run_name))

In [ ]:
# model = tf.keras.models.load_model("{}/models_saved/fcdnn_real.h5".format(run_name))

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
%%time
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))
y_true_array = []
y_pred_array = []
for j in tqdm(range(len(all_data)), "Options...", leave=False):
    y_true = []
    y_pred = []
    for energy in tqdm(energies, "Energies...", leave=False):
        test = pd.read_csv('{}/ProcessedTestReal/{}MeV.csv.gz'.format(path, energy))
        test = test[test['edepR'] < 17.2]
        edep = np.array(test['edep'])
        X_test = test[features_opt]
        X_test = scaler.transform(X_test)
        edep_preds = model.predict(X_test).flatten()
        
        y_true.append(edep)
        y_pred.append(edep_preds)
    y_true_array.append(y_true)
    y_pred_array.append(y_pred)

In [ ]:
from eli5.permutation_importance import get_score_importances 
from sklearn.metrics import mean_squared_error

def score(X, y):
    y_pred = model.predict(X)
    return mean_squared_error(y, y_pred)

base_score, score_decreases = get_score_importances(score, X[:50000], y[:50000])
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
fi = dict(zip(features_opt, feature_importances))
fi = dict(sorted(fi.items(), key=lambda item: item[1]))
df_fi = pd.DataFrame.from_dict(fi, orient='index', columns=['Permutation Importance'])
df_fi

In [ ]:
from neptunecontrib.api import log_table
log_table('output/permutation_importance', df_fi)

In [ ]:
diffs = np.array([
    [y_pred_array[j][i] - y_true_array[j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_pred_array))
])

In [ ]:
names = ['Elecsim 5M Real', 'Detsim 5M Ideal', 'Detsim 5M Real']

In [ ]:
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))
energies = np.array([1.022+i for i in energies]).round(5)
energies

In [ ]:
%%time
a_array = []
errors_array = []
for k in range(diffs.shape[0]):
    a = []
    e = []
    for i in range(diffs.shape[1]):
        fig, ax = plt.subplots()
        nbins = 150
        n, bins, patches = ax.hist(diffs[k][i], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before');
        plt.close(fig)
        centers = (0.5*(bins[1:]+bins[:-1]))
        pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[0,1])  
        a.append(pars)
        e.append(cov)
    a_array.append(a)
    errors_array.append(e)

In [ ]:
from neptunecontrib.api import log_chart

for k in range(len(all_data)):
    fig = go.Figure()

    for i in range(len(diffs[k])): 
        x = np.linspace(diffs[k][i][:35000].min(), diffs[k][i][:35000].max(), 100)
        p = stats.norm.pdf(x, a_array[k][i][0], a_array[k][i][1])

        fig.add_trace(go.Scattergl(x=x,
                     y=p, mode='lines', name='mu={:.3f} +- {:.3f}, sigma={:.3f} +- {:.3f}'.format(
                         a_array[k][i][0], np.sqrt(errors_array[k][i][0][0]),
                         a_array[k][i][1], np.sqrt(errors_array[k][i][1][1])),
                        visible = (i==0)
                    ))

    for i in range(len(diffs[k])): 
        fig.add_trace(go.Histogram(x=diffs[k][i][:35000], xbins=dict(size=0.005),
                      showlegend=False, histnorm='probability density',
                     visible = (i==0)
                    ))

    buttons = []
    for N in range(0, len(diffs[k])): 
        buttons.append(
            dict(
                 args=['visible', [False]*N + [True] + [False]*(len(diffs[k])-1-N)],
                     label='Energy =  {} MeV'.format(energies[N]),
                 method='restyle'
            )
        )
        
    fig.update_layout(

        xaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),

        yaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tick0=0,
#             dtick=1,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=0.25
            ),
    )

    fig.update_layout(
        title = '{}'.format(names[k]),
        xaxis_title=r"$$E_{rec} - E_{true}$$",
        showlegend=True,
        updatemenus=list([
            dict(
                x=0.5,
                y=1.2,
                yanchor='top',
                buttons=buttons
            ),
        ]),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()
    log_chart('output/Result_distributions.pdf', fig)
#     run['output/Result_distributions.pdf'].upload(File.as_html(fig))

In [ ]:
def plot_results(appr=False):
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01,
                        row_width=[0.25, 0.75]
    )

    for k in range(diffs.shape[0]):
        fig.add_trace(
            go.Scatter(
                x=energies,
                y=res[k],
                mode='markers',
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                showlegend=True,
                error_y=dict(
                    type='data',
                    width=10,
                    array=error_sigma[k],
                    visible=True
                ),
                name=names[k]
            ), row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=energies,
                y=bias[k],
                mode='markers',
                showlegend=False,
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                error_y=dict(
                        type='data',
                        width=10,
                        array=error_mu[k],
                        visible=True
                ),
                name=names[k]
            ), row=2, col=1
        )

    if appr:
        for k in range(len(names)):
            fig.add_trace(
                go.Scatter(
                    x=x_lin,
                    y=func(x_lin, a[k], b[k], c[k]),
                    mode='lines',
                    line=dict(
                    ),
                    opacity=0.5,
                    showlegend=False,
                    name=names[k],
                    marker=dict(
                        color=colors[k]
                    )
                ), row=1, col=1
            )

    trace = lambda x, y, error_y, col, name, sym, leg=True: go.Scatter(
                        x=x,
                        y=100*y,
                        mode='markers',
                        name=name,
                        showlegend=leg,
                        marker=dict(
                            color=col,
                            symbol=sym
                        ),
                        error_y=dict(
                            type='data',
                            width=10,
                            array=100*error_y
                        )
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_0.energy+1.022,
            bdt_J19_5M_0.res,
            bdt_J19_5M_0.res_err,
            'blue',
            'Detsim 5M Ideal',
            'square',
        ),
        row=1, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_23.energy+1.022,
            bdt_J19_5M_23.res,
            bdt_J19_5M_23.res_err,
            'green',
            'Detsim 5M Real',
            'cross-open-dot',
        ),
        row=1, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_0.energy+1.022,
            bdt_J19_5M_0.bias,
            bdt_J19_5M_0.bias_err,
            'blue',
            'Detsim 5M Ideal',
            'square',
            False
        ),
        row=2, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_23.energy+1.022,
            bdt_J19_5M_23.bias,
            bdt_J19_5M_23.bias_err,
            'green',
            'Detsim 5M Real',
            'cross-open-dot',
            False
        ),
        row=2, col=1
    )

    xaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        tick0=1,
        dtick=1,
        linecolor='black',
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
    )

    yaxis = lambda range: dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        range=range,
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=0.25
    )

    fig.update_layout(
        xaxis2_title="Visible energy, MeV",
        yaxis1_title="Resolution, %",
        yaxis2_title="Bias, %",

        xaxis1 = xaxis,
        xaxis2 = xaxis,
        yaxis1 = yaxis([0, 3.5]),
        yaxis2 = yaxis([-0.25, 0.25]),

        showlegend=True,
        font=dict(
                family="Times New Roman",
                size=18,
        ),
        legend=dict(
            x=0.75,
            y=0.99,
            title_font_family="Times New Roman",
            font=dict(
                family="Times New Roman",
                size=18,
                color="black"
            ),
            bordercolor="Black",
            borderwidth=2
        )
    )

    fig.show()
    if appr:
        pio.write_image(fig, '{}/results/appr_results.pdf'.format(run_name), width=900, height=600)
        log_chart('{}/results/appr_results.pdf'.format(run_name), fig)
#         run['output/appr_results.pdf'].upload(File.as_html(fig))
    else:
        pio.write_image(fig, '{}/results/results.pdf'.format(run_name), width=900, height=600)
        log_chart('{}/results/results.pdf'.format(run_name), fig)
#         run['output/results.pdf'].upload(File.as_html(fig))

In [ ]:
colors = ['darkviolet', 'blue', 'green']
symbols = ['star-square']

In [ ]:
error_sigma = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][1][1]) / energies[i] for i in range(len(energies))]
    error_sigma.append(error)
    
error_mu = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][0][0]) / energies[i] for i in range(len(energies))]
    error_mu.append(error)

res = []
bias = []
for k in range(diffs.shape[0]):
    sigma = [100 * a_array[k][i][1] / energies[i] for i in range(len(energies))]
    mu = [100 * a_array[k][i][0] / energies[i] for i in range(len(energies))]
    res.append(sigma)
    bias.append(mu)
    
plot_results()

In [ ]:
def a(x, a):
    return np.sqrt((a/x**0.5)**2)


def b(x, b):
    b_list = []
    b_list.append(np.sqrt(b**2))
    return b_list*len(x)


def c(x, c):
    return np.sqrt((c/x)**2)


def func(x, a, b, c):
    return np.sqrt((a/x**0.5)**2 + b**2 + (c/x)**2) 


def approximated(x, y, yerr):
    popt, pcov = curve_fit(func, x, y, sigma=yerr, maxfev=10**9, bounds=([0, 0, 0], [5, 5, 5]))
    a, b, c = popt
    #perr = np.sqrt(abs(pcov.diagonal()))

    return func(x, a, b, c), popt, pcov

In [ ]:
from statsmodels.stats.moment_helpers import cov2corr

y_approximated_array = []
coefs_array = []
errors_array = []
corr_matrixes = []
for i in range(diffs.shape[0]):
    y_approximated, coefs, pcov = approximated(
        energies[1:13], res[i][1:13], error_sigma[i][1:13])
    y_approximated_array.append(y_approximated)
    coefs_array.append(coefs)
    errors_array.append(np.sqrt(abs(pcov.diagonal())))
    corr_matrixes.append(cov2corr(pcov))

y_approximated, coefs, pcov = approximated(
    bdt_J19_5M_0.energy+1.022, 100*bdt_J19_5M_0.res, 100*bdt_J19_5M_0.res_err)
y_approximated_array.append(y_approximated)
coefs_array.append(coefs)
errors_array.append(np.sqrt(abs(pcov.diagonal())))
corr_matrixes.append(cov2corr(pcov))

y_approximated, coefs, pcov = approximated(
    bdt_J19_5M_23.energy+1.022, 100*bdt_J19_5M_23.res, 100*bdt_J19_5M_23.res_err)
y_approximated_array.append(y_approximated)
coefs_array.append(coefs)
errors_array.append(np.sqrt(abs(pcov.diagonal())))
corr_matrixes.append(cov2corr(pcov))

corr_matrixes = np.array(corr_matrixes)

In [ ]:
reindex = [0, 3, 1, 4, 2, 5]
coefs_df = pd.DataFrame(
    np.hstack((coefs_array, errors_array))
)[reindex]
coefs_df.columns = ['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$']

a = np.array(coefs_array).T[0]
b = np.array(coefs_array).T[1]
c = np.array(coefs_array).T[2]

In [ ]:
x_lin = np.linspace(0.8, 11.5, 1000)
plot_results(appr=True)

**The JUNO collaboration., Abusleme, A., Adam, T. et al. Calibration strategy of the JUNO experiment. J. High Energ. Phys. 2021, 4 (2021). https://doi.org/10.1007/JHEP03(2021)004**

It was found, numerically, that the JUNO baseline requirement to determine the MO to 3 – 4 $\sigma$ significance could be translated into a convenient requirement on an effective resolution $\tilde{a}$ as:

$$\tilde{a} \equiv \sqrt{(a)^2 + (1.6 \times b)^2 + \left(\frac{c}{1.6}\right)^2} \leq 3\%$$

Let's calculate the variance as follows:

$$\Delta \tilde{a}^2 = \left(\frac{\partial \tilde{a}}{\partial a} \Delta a \right)^2 +
                       \left(\frac{\partial \tilde{a}}{\partial b} \Delta b \right)^2 +
                       \left(\frac{\partial \tilde{a}}{\partial c} \Delta c \right)^2 +
2 \left[
\left(\frac{\partial \tilde{a}}{\partial a} \right) \left(\frac{\partial \tilde{a}}{\partial b} \right) r_{ab} \Delta a \Delta b +
\left(\frac{\partial \tilde{a}}{\partial a} \right) \left(\frac{\partial \tilde{a}}{\partial c} \right) r_{ac} \Delta a \Delta c +
\left(\frac{\partial \tilde{a}}{\partial b} \right) \left(\frac{\partial \tilde{a}}{\partial c} \right) r_{bc} \Delta b \Delta c
\right]$$

Then:

$$\Delta \tilde{a}^2 = \frac{\left(a\Delta a\right)^2 + \left(2.56\times b\Delta b\right)^2 + \left(\frac{c\Delta c}{2.56}\right)^2}{\tilde{a}^2} + 2 \left[
\frac{2.56ab}{\tilde{a}^2} r_{ab} \Delta a \Delta b +
\frac{ac}{2.56\tilde{a}^2} r_{ac} \Delta a \Delta c +
\frac{bc}{\tilde{a}^2} r_{bc} \Delta b \Delta c
\right] = 
\frac{1}{\tilde{a}^2} \left[
\left(a\Delta a\right)^2 + \left(2.56\times b\Delta b\right)^2 + \left(\frac{c\Delta c}{2.56}\right)^2 +
5.12 a b r_{ab} \Delta a \Delta b + 
\frac{ac}{1.28} r_{ac} \Delta a \Delta c +
2bcr_{bc} \Delta b \Delta c
\right]$$


In [ ]:
coefs_df = coefs_df.round(3)
coefs_df.index = names
coefs_df[r'$\tilde{a}$'] = (coefs_df['a']**2 + (1.6 * coefs_df['b'])**2 + (coefs_df['c'] / 1.6)**2)**0.5 
coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt( (coefs_df['a']*coefs_df[r'$\Delta a$'])**2 + \
                                           (2.56*coefs_df['b']*coefs_df[r'$\Delta b$'])**2 + \
                                           (coefs_df['c']*coefs_df[r'$\Delta c$'] / 2.56)**2) / coefs_df[r'$\tilde{a}$']

coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt(
    coefs_df[r'$\Delta \tilde{a}$']**2 + 2 * (
        1.6**2 * coefs_df['a'] * coefs_df['b'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 0, 1] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta b$'] +\
        
        coefs_df['a'] * coefs_df['c'] / (coefs_df[r'$\tilde{a}$']**2 * 1.6**2) *\
        corr_matrixes[:, 0, 2] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta c$'] +\

        coefs_df['b'] * coefs_df['c'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 1, 2] * coefs_df[r'$\Delta b$'] * coefs_df[r'$\Delta c$']
    )
)

In [ ]:
# run['output/coefs_df'].upload(File.as_html(coefs_df))
log_table('output/coefs_df', coefs_df)
coefs_df

In [ ]:
from neptune import log_metric
log_metric('a_tilde', coefs_df[r'$\tilde{a}$']['Elecsim 5M Real'])
log_metric('a_tilde_std', coefs_df[r'$\Delta \tilde{a}$']['Elecsim 5M Real'])

In [ ]:
coefs_df.reset_index().to_csv('{}/results/params.csv'.format(run_name), index=False)